# Running Dask on the cluster with MLrun

The dask frameworks enables you to parallelize your python code and run it as a distributed process on Iguazio cluster and dramatically accelerate the performance. <br>
In this notebook you'll learn how to create a dask cluster and then an mlrun function running as a dask client. <br>
It also demonstrates how to run parallelize custom algorithm using Dask Delayed option.

For more information on dask over kubernetes: https://kubernetes.dask.org/en/latest/.

## Set up the environment

In [1]:
# set mlrun api path and artifact path for logging
import mlrun
mlrun.get_or_create_project("dask-demo", "./")

> 2023-02-06 08:21:13,178 [info] loaded project dask-demo from MLRun DB


## Create and start Dask cluster
Dask functions can be local (local workers), or remote (use containers in the cluster). In the case of remote you 
can specify the number of replica (optional) or leave blank for auto-scale.<br>
Use the `new_function()`  to define the Dask cluster and set the desired configuration of that clustered function.

If the dask workers need to access the shared file system, apply a shared volume mount (e.g. via v3io mount).

The Dask function spec has several unique attributes (in addition to the standard job attributes):

* **.remote** &mdash; bool, use local or clustered dask
* **.replicas** &mdash; number of desired replicas, keep 0 for auto-scale
* **.min_replicas**, **.max_replicas** &mdash; set replicas range for auto-scale
* **.scheduler_timeout** &mdash; cluster is killed after timeout (inactivity), default is '60 minutes'
* **.nthreads** &mdash; number of worker threads
<br>

If you want to access the dask dashboard or scheduler from remote you need to use NodePort service type (set `.service_type` to 'NodePort'), and the external IP need to be specified in mlrun configuration (mlconf.remote_host). This is set automatically if you are running on an Iguazio cluster.

Specify the kind (dask) and the container image:

In [6]:
# create an mlrun function that will init the dask cluster
dask_cluster_name = "dask-cluster"
dask_cluster = mlrun.new_function(dask_cluster_name, kind='dask', image='mlrun/ml-models')
dask_cluster.apply(mlrun.mount_v3io())

In [7]:
# set range for # of replicas with replicas and max_replicas
dask_cluster.spec.min_replicas = 1
dask_cluster.spec.max_replicas = 4

# set the use of dask remote cluster (distributed) 
dask_cluster.spec.remote = True
dask_cluster.spec.service_type = "NodePort"

# set dask memory and cpu limits
dask_cluster.with_requests(mem='2G', cpu='2')

### Initialize the Dask Cluster

When you request the dask cluster `client` attribute, it verifies that the cluster is up and running:

In [42]:
# init dask client and use the scheduler address as param in the following cell
dask_cluster.client

> 2023-02-06 07:56:46,437 [info] trying dask client at: tcp://mlrun-dask-cluster-5ddb2dfa-0.default-tenant:8786
> 2023-02-06 07:56:46,486 [info] using remote dask scheduler (mlrun-dask-cluster-5ddb2dfa-0) at: tcp://mlrun-dask-cluster-5ddb2dfa-0.default-tenant:8786


Connection method: Direct,
Dashboard: http://mlrun-dask-cluster-5ddb2dfa-0.default-tenant:8787/status,
Comm: tcp://10.201.41.176:8786,Workers: 0
Dashboard: http://10.201.41.176:8787/status,Total threads: 0
Started: Just now,Total memory: 0 B


## Creating a function that runs over Dask

In [11]:
# mlrun: start-code

Import mlrun and dask. nuclio is only used to convert the code into an mlrun function.

In [12]:
import mlrun 

In [13]:
%nuclio config kind = "job"
%nuclio config spec.image = "mlrun/ml-models"

%nuclio: setting kind to 'job'
%nuclio: setting spec.image to 'mlrun/ml-models'


In [14]:
from dask.distributed import Client
from dask import delayed
from dask import dataframe as dd

import warnings
import numpy as np
import os
import mlrun

warnings.filterwarnings("ignore")

### python function code

This simple function reads a .csv file using dask dataframe. It runs the `groupby` and `describe` functions on the dataset, and stores the results as a dataset artifact.<br>

In [15]:
def test_dask(context,
              dataset: mlrun.DataItem,
              client=None,
              dask_function: str=None) -> None:
    
    # setup dask client from the MLRun dask cluster function
    if dask_function:
        client = mlrun.import_function(dask_function).client
    elif not client:
        client = Client()
    
    # load the dataitem as dask dataframe (dd)
    df = dataset.as_df(df_module=dd)
    
    # run describe (get statistics for the dataframe) with dask
    df_describe = df.describe().compute()
    
    # run groupby and count using dask 
    df_grpby = df.groupby("VendorID").count().compute()
    
    context.log_dataset("describe", 
                    df=df_grpby,
                    format='csv', index=True)
    return

In [16]:
# mlrun: end-code

## Test the function over Dask

### Load sample data

In [17]:
DATA_URL="/User/examples/ytrip.csv"

In [18]:
!mkdir -p /User/examples/
!curl -L "https://s3.wasabisys.com/iguazio/data/Taxi/yellow_tripdata_2019-01_subset.csv" > {DATA_URL}

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 84.9M  100 84.9M    0     0  17.3M      0  0:00:04  0:00:04 --:--:-- 19.1M


### Convert the code to MLRun function

Use `code_to_function` to convert the code to MLRun and specify the configuration for the dask process (e.g. replicas, memory etc.). <br>
Note that the resource configurations are per worker.

In [19]:
# mlrun transforms the code above (up to nuclio: end-code cell) into serverless function 
# which runs in k8s pods
fn = mlrun.code_to_function("test_dask",  kind='job', handler="test_dask").apply(mlrun.mount_v3io())

### Run the function

In [21]:
# function URI is db://<project>/<name>
dask_uri = f'db://{project_name}/{dask_cluster_name}'

In [22]:
r = fn.run(handler = test_dask,
           inputs={"dataset": DATA_URL},
           params={"dask_function": dask_uri})

> 2021-01-24 23:49:37,858 [info] starting run test-dask-test_dask uid=6410ec27b63e4a12b025696fcabc2dc9 DB=http://mlrun-api:8080
> 2021-01-24 23:49:38,069 [info] Job is running in the background, pod: test-dask-test-dask-rmgkn
> 2021-01-24 23:49:41,647 [warning] Unable to parse server or client version. Assuming compatible: {'server_version': 'unstable', 'client_version': 'unstable'}
> 2021-01-24 23:49:42,112 [info] using in-cluster config.
> 2021-01-24 23:49:42,113 [info] trying dask client at: tcp://mlrun-dask-cluster-b3c6e737-3.default-tenant:8786
> 2021-01-24 23:49:42,134 [info] using remote dask scheduler (mlrun-dask-cluster-b3c6e737-3) at: tcp://mlrun-dask-cluster-b3c6e737-3.default-tenant:8786
remote dashboard: default-tenant.app.yh57.iguazio-cd0.com:30433
> 2021-01-24 23:49:48,334 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
dask-demo,...cabc2dc9,0,Jan 24 23:49:42,completed,test-dask-test_dask,v3io_user=adminkind=jobowner=adminhost=test-dask-test-dask-rmgkn,dataset,dask_function=db://dask-demo/dask-cluster,,describe


to track results use .show() or .logs() or in CLI: 
!mlrun get run 6410ec27b63e4a12b025696fcabc2dc9 --project dask-demo , !mlrun logs 6410ec27b63e4a12b025696fcabc2dc9 --project dask-demo
> 2021-01-24 23:49:50,284 [info] run executed, status=completed


## Track the progress in the UI

You can view the progress and detailed information in the mlrun UI by clicking on the uid above. <br>
To track the dask progress: in the Dask UI click the "dashboard link" above the "client" section.